In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
# upload frame into loader selecting column for vectorization
DATA = os.getenv('DATA')

In [13]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

In [5]:
# Load the documents from the directory and change the text loader to use the autodetect_encoding option
text_loader_kwargs={'autodetect_encoding': True} # this is a dictionary of keyword arguments to pass to the loader_cls
loader = DirectoryLoader(DATA, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs) 
# **/*.txt means all files with .txt extension in all subdirectories
# loader_kwargs is a dictionary of keyword arguments to pass to the loader_cls

In [6]:
docs = loader.load()
print(f'Loaded {len(docs)} doc')

Loaded 137 doc


In [7]:
docs[0]

Document(page_content='title: Energía | Scalian Spain\nurl: https://www.scalian-spain.es/experiencias-de-negocio/energia/\n\n\n\nEnergía\n\n\nAyudamos a revolucionar el modo en que las empresas energéticas gestionan los datos para ofrecer mejores servicios al usuario y ser más eficientes en la toma de decisiones.\n\n\nAprender de los datos para reinventarse.\n\n\nCompañías de energía conviven en un entorno sumamente competitivo donde la transformación es la clave que les permitirá\xa0 aumentar su rentabilidad y destacar. Ya sean Productoras, Distribuidoras o Comercializadoras»\xa0 todas deben acelerar su proceso de digitalización que les permita una mejor toma de decisiones basadas en sus propios datos. Una realidad que ya ha empezado y no pueden seguir aplazando.\n\n\nOptimización de procesos complejos  para mejorar la eficiencia.\n\n\nLos trabajos en campo, tanto para labores de mantenimiento como para labores de operación o lectura, son uno de los elementos de mayor coste para las c

In [44]:
# split the documents into chunks of 2500 characters with no overlap
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
print(texts)

Created a chunk of size 1313, which is longer than the specified 1000
Created a chunk of size 12151, which is longer than the specified 1000
Created a chunk of size 4762, which is longer than the specified 1000
Created a chunk of size 1021, which is longer than the specified 1000
Created a chunk of size 3949, which is longer than the specified 1000
Created a chunk of size 1246, which is longer than the specified 1000
Created a chunk of size 3582, which is longer than the specified 1000
Created a chunk of size 1785, which is longer than the specified 1000
Created a chunk of size 1555, which is longer than the specified 1000
Created a chunk of size 1585, which is longer than the specified 1000
Created a chunk of size 1162, which is longer than the specified 1000
Created a chunk of size 1016, which is longer than the specified 1000
Created a chunk of size 1380, which is longer than the specified 1000
Created a chunk of size 2347, which is longer than the specified 1000
Created a chunk of 

[Document(page_content='title: Energía | Scalian Spain\nurl: https://www.scalian-spain.es/experiencias-de-negocio/energia/\n\nEnergía\n\n\nAyudamos a revolucionar el modo en que las empresas energéticas gestionan los datos para ofrecer mejores servicios al usuario y ser más eficientes en la toma de decisiones.\n\n\nAprender de los datos para reinventarse.\n\n\nCompañías de energía conviven en un entorno sumamente competitivo donde la transformación es la clave que les permitirá\xa0 aumentar su rentabilidad y destacar. Ya sean Productoras, Distribuidoras o Comercializadoras»\xa0 todas deben acelerar su proceso de digitalización que les permita una mejor toma de decisiones basadas en sus propios datos. Una realidad que ya ha empezado y no pueden seguir aplazando.\n\n\nOptimización de procesos complejos  para mejorar la eficiencia.', metadata={'source': '/Users/gala/PycharmProjects/Scalian_Chat/ScalianChatBot/Data/spain/www_scalian-spain_es_experiencias-de-negocio_energia.txt'}), Document

In [47]:
# create a vector store using the OpenAI embeddings
openai_api_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [48]:
# create a vector store using the OpenAI embeddings and Chroma vector store
docsearch = Chroma.from_documents(texts, embeddings)
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=openai_api_key, temperature=0), 
    chain_type="stuff", 
    retriever=docsearch.as_retriever()
)

In [49]:
docsearch.similarity_search_with_score("What strategies does Scalian Spain employ to ensure a smooth transition to new operational management paradigms, and how do they integrate digitalization, performance management, and expertise to achieve this goal?")

[(Document(page_content='title: Operations Performance | Scalian Spain\nurl: https://www.scalian-spain.es/especializacion/operations-performance/\n\nOperations Performance\n\n\nCombinamos la capacidad de digitalización de procesos y transformación digital junto con nuestra aptitud de gestión del rendimiento de las operaciones para asegurar una transición segura a nuevos paradigmas de gestión de las operaciones.\n\n\nImpulsando el negocio.\n\n\nEn Scalian Spain somos expertos en ayudar a las empresas a establecer sinergias entre sus distintas unidades productivas y departamentos dotándolos de las herramienta y los servicios que que les permitan incrementar de forma considerable su rendimiento y su capacidad operativa para producir conjuntamente de forma que se consigan los objetivos empresariales que se propongan al tiempo que se mejora la capacidad de respuesta, el rendimiento, la calidad, el coste y la eficiencia de sus sistemas de producción o servicios.\n\n\nPlay Video\n\n\nOperacio

In [50]:
def query(q):
    print("Query: ", q)
    print("Answer: ", qa.invoke(q))

In [51]:
query("What strategies does Scalian Spain employ to ensure a smooth transition to new operational management paradigms, and how do they integrate digitalization, performance management, and expertise to achieve this goal")

Query:  What strategies does Scalian Spain employ to ensure a smooth transition to new operational management paradigms, and how do they integrate digitalization, performance management, and expertise to achieve this goal
Answer:  {'query': 'What strategies does Scalian Spain employ to ensure a smooth transition to new operational management paradigms, and how do they integrate digitalization, performance management, and expertise to achieve this goal', 'result': ' Scalian Spain combines digitalization of processes and digital transformation with their expertise in performance management to ensure a safe transition to new operational management paradigms. They also integrate the use of technology, such as data collection and analysis, and the expertise of their team to optimize processes and improve operational intelligence. Additionally, they offer services in asset management to improve reliability and minimize risk and costs.'}


In [52]:
query("Does  'scalian' organize workshops and werbinars?")

Query:  Does  'scalian' organize workshops and werbinars?
Answer:  {'query': "Does  'scalian' organize workshops and werbinars?", 'result': ' It is not mentioned in the given context whether Scalian organizes workshops and webinars.'}


In [53]:
query('Qui est le PDG du groupe Scalian ?')

Query:  Qui est le PDG du groupe Scalian ?
Answer:  {'query': 'Qui est le PDG du groupe Scalian ?', 'result': ' Yvan Chabanne'}


In [54]:
query('Miembros del equipo de gestión de Scalian?')

Query:  Miembros del equipo de gestión de Scalian?
Answer:  {'query': 'Miembros del equipo de gestión de Scalian?', 'result': ' No se mencionan específicamente miembros del equipo de gestión de Scalian en el contexto proporcionado.'}


In [55]:
query('Miembros del equipo de gestión de Scalian con sus titulos?')

Query:  Miembros del equipo de gestión de Scalian con sus titulos?
Answer:  {'query': 'Miembros del equipo de gestión de Scalian con sus titulos?', 'result': '\n1. Yvan Chabanne - Presidente del Grupo Scalian\n2. Jean-Charles de Borda - Chief Operations Officer Europe\n3. Fanny Mercieca - Mergers & Acquisitions Director\n4. Christophe Reybert - Chief Operations Officer France\n5. David Capdaspe Lagangue - Director Administrativo y Financiero\n6. Caroline Nancy - Directora de Recursos Humanos\n7. Jean-Manuel Silva - Secretario General y Director Jurídico\n8. Marie de Saint Salvy - Senior Vice-President and member of the Executive Committee'}
